In [1]:
import re
import numpy as np
import pandas as pd
import math
import pickle
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [2]:
stopwords=['కూడా', 'ఉన్నారు', 'ఎవరైనా', 'ఎప్పుడు', 'ఎక్కడైనా', 'అందుబాటులో', 'ఒక ప్రక్కన', 'సంబంధం', 'మాత్రమే', 'అడగండి', 'గా', 'మధ్య', 'ప్రకారం', 'అనుమతించు', 'మెచ్చుకో', 'వద్ద', 'తగిన', 'అడ్డంగా', 'అడగడం', 'ఇప్పటికే', 'కనిపిస్తాయి', 'కాదు', 'మరియు', 'మరొక', 'అనుగుణంగా', 'అయితే', 'చేయగలిగింది', 'దాదాపు', 'గురించి', 'పై', 'వెంట', 'వేరుగా', 'చుట్టూ', 'దూరంగా', 'ఏ', 'నిజంగా', 'అనుమతిస్తుంది', 'అందరూ', 'ఎవరో ఒకరు', 'వ్యతిరేకంగా', 'ఏదైనా', 'తర్వాత', 'మళ్ళీ', 'ఏమైనప్పటికి', 'ఒక']

In [3]:
vowels=set("అ ఆ ఇ ఈ ఉ ఊ ఋ ౠ ఌ ౡ ఎ ఏ ఐ ఒ ఓ ఔ అం అః".split())

conso="క	ఖ	గ	ఘ	ఙ	చ	ఛ	జ	ఝ	ఞ	ట	ఠ	డ	ఢ	ణ	త	థ	ద	ధ	న	ప	ఫ	బ	భ	మ	య	ర	ల	వ	శ	ష	స	హ	ళ	క్ష	ఱ"

matra = "ఀ	ఁ	ం	ః	ఄ	ఽ	ా	ి	ీ	ు	ూ	ృ	ౄ	ె	ే	ై	ొ	ో	ౌ	్	ౕ	ౖ	ౢ	ౣ"

halantha='్'

In [4]:
consonants=set(conso.split("\t"))
matra = set(matra.split("\t"))

vowels = list(vowels)
consonants=list(consonants)
matra=list(matra)

alphabets = vowels+consonants

In [5]:
print(alphabets)

['ఐ', 'అః', 'ఓ', 'ఈ', 'ఒ', 'అం', 'ఏ', 'ౠ', 'ఌ', 'ఆ', 'ఉ', 'ఋ', 'ఇ', 'అ', 'ఊ', 'ఔ', 'ౡ', 'ఎ', 'హ', 'థ', 'ఱ', 'భ', 'ఖ', 'ఢ', 'వ', 'శ', 'స', 'క్ష', 'ణ', 'ఘ', 'త', 'గ', 'ష', 'న', 'జ', 'ర', 'ఠ', 'ల', 'ఙ', 'చ', 'ళ', 'ఞ', 'ఛ', 'ప', 'ఝ', 'క', 'ఫ', 'ధ', 'య', 'బ', 'ట', 'ద', 'మ', 'డ']


In [7]:
data=pd.read_csv('../telugu.csv')

In [8]:
def remove_special_characters(text):
    string=''
    for i in text:
        if i in alphabets+matra:
            string=string+i
        else:
            string=string+" "
    return " ".join(string.split())

In [9]:
with open('pickles/posting_list.pkl','rb') as file:
    tf=pickle.load(file)
    file.close()
    
with open('pickles/df.pkl','rb') as file:
    DF=pickle.load(file)
    file.close()
    
with open('pickles/file_idx.pkl','rb') as file:
    file_idx=pickle.load(file)
    file.close()
    
with open('pickles/doc_len.pkl','rb') as file:
    doc_len=pickle.load(file)
    file.close()

In [10]:
k=0
Ld=doc_len
N=len(file_idx)
for i in Ld:
    k+=Ld[i]
Lavg=k/N

In [11]:
def IDF(q):
    DF1=0
    if q in DF:
        DF1=DF[q]
    ans=math.log((N-DF1+0.5)/(DF1+0.5))
    return ans
k=1.2
b=0.75

In [12]:
def bm25(query,counter=10):
    score={}
    for i in range(len(file_idx)):
        score[i]=0
    text = remove_special_characters(query)
    words = text.split()
    words=[word for word in words if word not in stopwords]
    words=[word for word in words if word in DF.keys()]
    for i in range(len(file_idx)):
        score[i]=0
        for qi in words:
            TF=0
            if qi in tf:
                if i in tf[qi]:
                    TF=tf[qi][i]
            idf=IDF(qi)
            ans=idf*(k+1)*TF/(TF+k*(1-b+b*(Ld[i]/Lavg)))
            score[i]+=ans
    score=sorted(score.items(),key=lambda item: item[1],reverse=True)
    verse_index=[]
    for i in score:
        if counter== 0:
            break
        verse_index.append(i[0])
        counter-=1
    return verse_index

In [13]:
verse_index=bm25('మత తగాదాలు')
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
142,3,31,ये मे मतमिदं नित्यमनुतिष्ठन्ति मानवाः।\n\nश्रद...,"చాలా అందంగా, దేవదేవుడు తను వివరించిన సిద్ధాంతా..."
473,12,12,श्रेयो हि ज्ञानमभ्यासाज्ज्ञानाद्ध्यानं विशिष्य...,చాలా మంది జనులు యాంత్రికమైన అభ్యాసము చేసే దశ ల...
298,7,26,वेदाहं समतीतानि वर्तमानानि चार्जुन।\n\nभविष्या...,భగవంతుడు సర్వజ్ఞుడు. తాను ‘త్రికాల-దర్శి’ అని ...
374,10,10,तेषां सततयुक्तानां भजतां प्रीतिपूर्वकम्।\n\nदद...,మన సొంత తెలివితేటలతో భగవంతుని యొక్క దివ్య జ్ఞా...
500,13,19,इति क्षेत्रं तथा ज्ञानं ज्ञेयं चोक्तं समासतः।म...,శ్రీ కృష్ణుడు ఇక ఇప్పుడు క్షేత్రమును మరియు జ్ఞ...
124,3,13,यज्ञशिष्टाशिनः सन्तो मुच्यन्ते सर्वकिल्बिषैः।\...,"వైదిక సాంప్రదాయం లో, అన్నము (ఆహారము) ను భగవంతు..."
318,8,16,आब्रह्मभुवनाल्लोकाः पुनरावर्तिनोऽर्जुन।\n\nमाम...,"వేద శాస్త్రములు, భూలోకం కన్నా నిమ్న స్థాయిలో ఏ..."
0,1,1,धृतराष्ट्र उवाच\n\nधर्मक्षेत्रे कुरुक्षेत्रे स...,ధృతరాష్ట్ర మహారాజు పుట్టుకతోనే గుడ్డి వాడే కాక...
1,1,2,सञ्जय उवाच\n\nदृष्ट्वा तु पाण्डवानीकं व्यूढं द...,తన పుత్రులు ఎలాగైనా యుద్ధం మొదలు పెడతారనే ధ్రు...
2,1,3,पश्यैतां पाण्डुपुत्राणामाचार्य महतीं चमूम्।\n\...,తన అస్త్రవిద్యా గురువు ద్రోణాచార్యుడికి గతంలో ...


In [15]:
data.loc[473]["Commentary"]

'చాలా మంది జనులు యాంత్రికమైన అభ్యాసము చేసే దశ లో ఉన్నారు. తమతమ మత/ఆచారాలలో చెప్పినట్టుగా కర్మకాండలను చేస్తుంటారు, కానీ భగవంతుని యందు మనస్సు లగ్నం చేయరు. కొత్త ఇల్లు లేదా కొత్త కారు కొన్నప్పుడు, పూజారిని పిలిచి పూజాది కార్యక్రమాలు చేస్తుంటారు. పూజారి పూజ చెస్తూఉన్నప్పుడు, వారు వేరే గదిలో కూర్చుని ఎదో మాట్లడుకోవటమో లేదా కప్పు టీ తాగటమో చేస్తుంటారు. వారికి భక్తి అంటే ఏదో యాంత్రికమైన కార్యము చేయటమే. తరచుగా అది ఆచార అలవాటుగా, తల్లితండ్రుల నుండి పెద్దల నుండి అందించబడుతుంది. యాంత్రికముగా పూజాది కార్యక్రమాలు చేయటం అనేది ఏమీ చెడు పని కాదు, ఎందుకంటే, ఏమీ చేయక పోవటం కన్నా ఎదో ఒకటి చేయటం మంచిది. కనీసం, బాహ్యంగా నైనా భక్తిలో నిమగ్నమవుతారు.   కానీ, శ్రీ కృష్ణుడు ఇక్కడ ఏమంటున్నాడంటే, ఈ యాంత్రికమైన అభ్యాసము కన్నా ఆధ్యాత్మిక జ్ఞానము పెంపొందించుకొనుట మంచిది. జీవితపు లక్ష్యము భగవత్ ప్రాప్తి అని, భౌతిక ప్రాపంచిక పురోగతి కాదన్న విషయాన్ని ఆధ్యాత్మిక జ్ఞానము ప్రసాదిస్తుంది. జ్ఞానమును బాగా తెలుసుకున్న వ్యక్తి, కేవలం యాంత్రిక కర్మ కాండల కంటే అంతఃకరణ శుద్ధి కోసమే వాంఛ పెంచుకుంటాడు. కానీ, కేవలం జ్ఞానం కూడా అంతఃక